In [332]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd
import time
from datetime import datetime
pd.set_option("display.max_colwidth",50)

In [29]:
Start = "https://www.presseportal.de/blaulicht/nr/4970"
html = urlopen(Start)
bs = BeautifulSoup(html, 'html.parser')
Links=bs.find_all("h2", class_="news-headline news-headline-clamp" )

In [30]:
#Try Pages not found again
def getPagesNotfound(AllPagesNotfound):
    
    LinksNotFoundBefore=[]
    LinksStillNotFOund=[]
    for SinglePage in AllPagesNotfound:
        try:
            bs = BeautifulSoup(SinglePage, 'html.parser')
            Tag=bs.find(class_="pagination-next")
            
            LinksNotFoundBefore.append("https://www.presseportal.de"+Tag.attrs['data-url'][1:])
            time.sleep(.3)
        except:
            time.sleep(1)
            LinksStillNotFOund.append("https://www.presseportal.de"+Tag.attrs['data-url'][1:])
    return LinksNotFoundBefore, LinksStillNotFOund
 
    
#getallsites
Start = "https://www.presseportal.de/blaulicht/nr/4970"
def getallsites(Start, MaxP):
    Pages=[]
    PagesNotfound=[]
    html = urlopen(Start)
    Success =0
    NoSuccess=0
    for i in range(1,MaxP+1):
        try:
            bs = BeautifulSoup(html, 'html.parser')
            Tag=bs.find(class_="pagination-next")
            Pages.append("https://www.presseportal.de"+Tag.attrs['data-url'][1:])
            html = urlopen("https://www.presseportal.de"+Tag.attrs['data-url'][1:])
            time.sleep(.3)
            Success +=1
        except:
            time.sleep(1)
            NoSuccess +=1
            PagesNotfound.append("https://www.presseportal.de"+Tag.attrs['data-url'][1:])
        time.sleep(.3)
        print("getallsites Progress "+str(round(((i/MaxP)*100),2))+"%" + " Success: " +str(Success) + " NoSuccess: "+str(NoSuccess), flush=True,end='\r')
    LinksNotFoundBefore,LinksStillNotFOund = getPagesNotfound(PagesNotfound)
    
    for Link in LinksNotFoundBefore:
        Pages.append(Link)
    
    return Pages,LinksStillNotFOund


In [89]:
AllSites, SitesNotFound =getallsites(Start,20)

In [90]:
def getCorpusFromLink(AllSites):
    MainsiteCorpus=[]
    RemainingSites= []
    for Site in AllSites:
        try:
            html = urlopen(Site)
            bs = BeautifulSoup(html, 'html.parser')
            Links=bs.find_all("h3", class_="news-headline-clamp" )
            MainsiteCorpus.append(Links)
        except:
            RemainingSites.append(Site)
            
    return MainsiteCorpus, RemainingSites

def getLinksFromCorpus(MainsiteCorpus):
    Linklist =[]
    for Site in MainsiteCorpus:
        for Headline in Site:
            Article= (Headline.findChild("")["href"])
            Linklist.append(Article)
    return Linklist

def findMainArticleForGivenText(Corpus):
    s = ''
    for element in Corpus:
        s += '\n' + ''.join(element.findAll(text = True))
        s
    start = 'Polizeipräsidium Frankfurt am Main'
    end = 'Rückfragen bitte an:'
    s = (s[s.find(start)+len(start):s.rfind(end)])
    return s

In [ ]:
def getNewsFromLinks(Liste):
    counter=0
    counter2=0
    News=[]
    Headline=[]
    Hauptartikel=[]
    LinkNotfound=[]
    for Link in Liste:
        try:
            html=urlopen('https://www.presseportal.de{}'.format(Link))
            bs = BeautifulSoup(html, 'html.parser')
            Ueberschrift= bs.find("h1").get_text()
            WholeText =bs.find_all('p')
            MainArticle =findMainArticleForGivenText(WholeText)
            Headline.append(Ueberschrift)
            Hauptartikel.append(MainArticle)
            counter+=1
            News.append('https://www.presseportal.de{}'.format(Link))
            time.sleep(.3)
            print("found-> "+ str(counter)+ "of "+ str(len(Liste)),flush=True,end='\r')
        except:
            counter2+=1
            LinkNotfound.append(Link)
            print("not found -> " + str(counter2)+ " of "+ str(len(Liste)),flush=True,end='\r')
        
    return Headline,Hauptartikel,News,LinkNotfound

In [93]:
Corpus, Other =getCorpusFromLink(AllSites)

In [94]:
LinksForAllNews = getLinksFromCorpus(Corpus)

In [286]:
#AlleHeadlines, AlleHauptartikel, AlleWebsiten, LinkNotfound =getNewsFromLinks(LinksForAllNews)

In [322]:
df = pd.DataFrame({'Headline':AlleHeadlines,'Hauptartikel':AlleHauptartikel,'Link':AlleWebsiten})

In [323]:
#Replace escape sharacters
df['Headline']= df["Headline"].apply(lambda st: st.replace("\t"," "))
df['Headline']= df["Headline"].apply(lambda st: st.replace("\n"," "))
df['Hauptartikel']= df["Hauptartikel"].apply(lambda st: st.replace("\n"," "))

df["date"] =df.Headline.str.extract('(\d+)')
df['date'] = df['date'].apply(lambda x: "{}{}".format('20', x))
#ffill if date is not found
df['date'] = df['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d') if len(x)==8 else "").fillna(method='ffill')
df['Headline'] = df['Headline'].str.replace('\d+', '')

In [324]:
#Extract case
#df["case"] =df.Headline.str.extract('(\s\d{3,4}\s)')
#df["case"] = df["case"].astype(str)

In [325]:
#Find index of Case in Headline
#df['col3'] = [i2.index(i1) for i1,i2 in zip(df.case,df.Headline)]
#COnvert into numeric
#df['col3'] = pd.to_numeric(df['col3'] )
#Remove beginning of Headline
#df['Headline'] = [A[B:] for A, B in zip(df["Headline"], df["col3"])]

In [326]:
#Strip whitespaces
df['Headline']= df["Headline"].apply(lambda st: st.lstrip())
#Remove beginning from Headline (remove 4digit Number)
df['Headline']=df['Headline'].str.replace("\d{4}","",regex=True)
df['Headline']=df['Headline'].str.replace("\d{3}","",regex=True)
#Strip whitespaces again
df['Headline']= df["Headline"].apply(lambda st: st.lstrip())

In [328]:
#Function to find nth substring
def find_nth(s, substr, n):
    i = 0
    while n >= 0:
        n -= 1
        i = s.find(substr, i + 1)
    return i

In [329]:
#Extract Location
df['Location']= df["Headline"].apply(lambda st: st[0:st.find(":")])
#Remove Headline
df['Headline']= df["Headline"].apply(lambda st: st[st.find(":")+1:])
#Strip whitespace again
df['Headline']= df["Headline"].apply(lambda st: st.lstrip())
#Extract second location
df['Location2']= df['Location'].str.split('/').str[1]
#Clean Location
df['Location']= df['Location'].str.split('/').str[0]
#Remove start of Hauptartikel e.g. Frankfurt (ots) - (ka)
df["Headline"].apply(lambda st: st[st.find(" - ")+2:find_nth(st," ",3)])
#Extract start of Hauptartikel
df["Beschreibung"]= df["Hauptartikel"].apply(lambda st: st[0:find_nth(st,")",1)+1])
#Remove start from Hauptartikel
df["Hauptartikel"]= df["Hauptartikel"].apply(lambda st: st[find_nth(st,")",1)+1:])

In [330]:
df=df.drop_duplicates()
df.count()

Headline        540
Hauptartikel    540
Link            540
date            540
Location        540
Location2       0  
Beschreibung    540
dtype: int64

In [126]:
df.to_csv("Test.csv",sep="@",encoding="UTF-16")

In [333]:
df

,Headline,Hauptartikel,Link,date,Location,Location2,Beschreibung
0,- Frankfurt-Sossenheim: Auto geklaut - Polize...,Dienstagabend (09.04.2019) meldeten Zeugen ei...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-11,POL-F,NaN,Frankfurt (ots) (pe)
1,- Frankfurt-Bornheim: Trickdiebin bringt Seni...,Am Dienstag (09.04.2019) haben Trickdiebe die...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-11,POL-F,NaN,Frankfurt (ots) (pe)
2,- Frankfurt-Eschersheim: Frau schlägt Räuber ...,In der vergangenen Nacht (10.-11.04.2019) hab...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-11,POL-F,NaN,Frankfurt (ots) (pe)
3,- Frankfurt-BAB: Schwerer Verkehrsunfall mit ...,Heute Vormittag (10.04.2019) kam es auf der A...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-10,POL-F,NaN,Frankfurt (ots) (em)
4,- Frankfurt-Praunheim: Rennradfahrer fährt F...,Am gestrigen Mittag (09.04.2019) fuhr ein Rad...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-10,POL-F,NaN,Frankfurt (ots) (pe)
5,- Frankfurt: Fälle sexueller Belästigungen - ...,"Wie den Medien bereits bekannt ist, kam es se...",https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-10,POL-F,NaN,Frankfurt (ots) (em)
6,- Frankfurt-Bahnhofsviertel: Parfumplagiat im...,Am gestrigen Abend (09.04.2019) hat die Frank...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-10,POL-F,NaN,Frankfurt (ots) (pe)
7,- Frankfurt-Bahnhofsviertel: Autodieb ermitte...,In der Nacht von Dienstag (09.04.2019) auf Mi...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-10,POL-F,NaN,Frankfurt (ots) (pe)
8,- Frankfurt-Innenstadt: -Jähriger baut Unfall...,In der Nacht von Sonntag auf Montag krachte e...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-09,POL-F,NaN,Frankfurt (ots) (ka)
9,- Frankfurt: Einladung zur Präventionsveranst...,Wer kennt es nicht? Schnell werden neue Schuh...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-09,POL-F,NaN,Frankfurt (ots) (em)


In [87]:
df["Clean_Hauptartikel"] = df['Hauptartikel'].str.replace('[^\w\s]','')

,Headline,Hauptartikel,Link,date,case,col3,Clean_Hauptartikel
0,Frankfurt-Sossenheim: Auto geklaut - Polizei e...,Frankfurt (ots) (pe) Dienstagabend (09.04.2...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-11,413,15,Frankfurt ots pe Dienstagabend 09042019 mel...
1,Frankfurt-Bornheim: Trickdiebin bringt Seniori...,Frankfurt (ots) (pe) Am Dienstag (09.04.201...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-11,412,15,Frankfurt ots pe Am Dienstag 09042019 haben...
2,Frankfurt-Eschersheim: Frau schlägt Räuber in ...,Frankfurt (ots) (pe) In der vergangenen Nac...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-11,411,15,Frankfurt ots pe In der vergangenen Nacht 1...
3,Frankfurt-BAB3: Schwerer Verkehrsunfall mit ei...,Frankfurt (ots) (em) Heute Vormittag (10.04...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-10,410,15,Frankfurt ots em Heute Vormittag 10042019 k...
4,Frankfurt-Praunheim: Rennradfahrer fährt Fußg...,Frankfurt (ots) (pe) Am gestrigen Mittag (0...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-10,409,15,Frankfurt ots pe Am gestrigen Mittag 090420...
5,Frankfurt: Fälle sexueller Belästigungen - Pol...,Frankfurt (ots) (em) Wie den Medien bereits...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-10,408,15,Frankfurt ots em Wie den Medien bereits bek...
6,Frankfurt-Bahnhofsviertel: Parfumplagiat im We...,Frankfurt (ots) (pe) Am gestrigen Abend (09...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-10,407,15,Frankfurt ots pe Am gestrigen Abend 0904201...
7,Frankfurt-Bahnhofsviertel: Autodieb ermittelt ...,Frankfurt (ots) (pe) In der Nacht von Diens...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-10,406,15,Frankfurt ots pe In der Nacht von Dienstag ...
8,Frankfurt-Innenstadt: 31-Jähriger baut Unfall ...,Frankfurt (ots) (ka) In der Nacht von Sonnt...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-09,405,15,Frankfurt ots ka In der Nacht von Sonntag a...
9,Frankfurt: Einladung zur Präventionsveranstalt...,Frankfurt (ots) (em) Wer kennt es nicht? Sc...,https://www.presseportal.de/blaulicht/pm/4970/...,2019-04-09,404,15,Frankfurt ots em Wer kennt es nicht Schnell...
